### rapyuta.io SDK turtlesim example.
Deploy turtlesim tutorial from SDK.

turtlesim tutorial: https://userdocs.rapyuta.io/quick-walkthrough/

SDK doc: https://docs.rapyuta.io/python-sdk/

#### 1.Auth, Parameter and File psths.
you can get auth from rapyuta.io console

SDK auth token:https://userdocs.rapyuta.io/developer-guide/tooling-automation/python-sdk/sdk-tokens-parameters/

In [ ]:
AUTH_TOKEN = ''
PROJECT_ID = ''

#params
DEPLOYMENT_PREFIX = 'turtlesim_'
TURTLE_NUM = 2 #max=2
retry_count=90
interval=10

#file paths of package manifests
base_path = '../io_manifests/turtlesim'
com_manifest = 'command_center_manifest.json'
sim_manifest = 'simulator_manifest.json'
turtle_manifest = 'turtle_manifest.json'
ui_manifest = 'user_interface_manifest.json'

#### 2. import module and create Client Instance

In [ ]:
from rapyuta_io import Client, DeploymentPhaseConstants, DiskType
from rapyuta_io.clients.package import ROSDistro

client = Client(AUTH_TOKEN, PROJECT_ID)

#### 3. Create routed network

In [ ]:
networks = client.get_all_routed_networks()
network = None
network_name = DEPLOYMENT_PREFIX + 'route_network'
for net in networks:
    if net.name == network_name and net.get_status().status == "Running":
        network = net
        print(DEPLOYMENT_PREFIX + 'route_network' + " was found and will be used")
        break
        
if not network:
    network = client.create_cloud_routed_network(network_name, ROSDistro.MELODIC, True)
    network.poll_routed_network_till_ready(retry_count=retry_count, sleep_interval=interval)
    print(DEPLOYMENT_PREFIX + 'route_network' + "was created successfully")

routed_networks = [network]

#### 4. Create package

In [ ]:
import os
import json
import time
from pprint import pprint

def create_package(file_path, packages):
    json_data = open(file_path, 'r')
    json_load = json.load(json_data)
    package_id = None
    for package in packages:
        if package['packageName'] == json_load['name'] and  \
           package['packageVersion'] == json_load['packageVersion']:
            package_id = package['packageId']
            break
    if not package_id:
        res = client.create_package(json_load, retry_limit=3)
        print(json_load['name'] + json_load['packageVersion'] + ' was created')
        package_id = res['packageId']
    else :
        print(json_load['name'] + json_load['packageVersion'] + ' exist')
    
    return package_id

def get_plan_id(pakcage_id, index=0):
    pack = client.get_package(pakcage_id)
    return pack['plans'][index]['planId']

def get_ids(base_path, manifest, packages):
    file_path = os.path.join(base_path, manifest)
    package_id = create_package(file_path, packages)
    plan_id = get_plan_id(package_id) 
    return {
        'packageId': package_id,
        'planId': plan_id        
    }

def wait_till_package_ready(package_id, retry_count=15, sleep_interval=6):
    count = 0
    while client.get_package(package_id)['status'] != 'Complete':
        time.sleep(sleep_interval)
        count+=1
        if count > retry_count:
            return False
    return True


packages = client.get_all_packages()
# pprint(packages)

sim_ids =    get_ids(base_path, sim_manifest     ,packages)    
com_ids =    get_ids(base_path, com_manifest     ,packages)    
turtle_ids = get_ids(base_path, turtle_manifest  ,packages)    
ui_ids =     get_ids(base_path, ui_manifest      ,packages)    

#### 3.Simulator deployment

In [ ]:
if not wait_till_package_ready(sim_ids['packageId'], retry_count, interval):
    exit(0)
simulator_package = client.get_package(sim_ids['packageId'])
simulator_configuration = simulator_package.get_provision_configuration(sim_ids['planId'])
simulator_configuration.add_routed_networks(routed_networks)
simulator_deployment = simulator_package.provision(
                deployment_name = DEPLOYMENT_PREFIX + 'simulator',
				provision_configuration = simulator_configuration)
res = simulator_deployment.poll_deployment_till_ready(retry_count=retry_count, sleep_interval=interval)

#### 4.Command Center deployment

In [ ]:
if not wait_till_package_ready(com_ids['packageId'], retry_count, interval):
    exit(0)
command_center_package = client.get_package(com_ids['packageId'])
command_center_configuration = command_center_package.get_provision_configuration(com_ids['planId'])
command_center_configuration.add_routed_networks(routed_networks)
command_center_deployment = command_center_package.provision(
                deployment_name = DEPLOYMENT_PREFIX + 'command_center',
				provision_configuration = command_center_configuration)
res = command_center_deployment.poll_deployment_till_ready(retry_count=retry_count, sleep_interval=interval)

#### 5.UI deployment

In [ ]:
if not wait_till_package_ready(ui_ids['packageId'], retry_count, interval):
    exit(0)
ui_package = client.get_package(ui_ids['packageId'])
ui_configuration = ui_package.get_provision_configuration(ui_ids['planId'])
ui_configuration.add_dependent_deployment(command_center_deployment)
ui_deployment = ui_package.provision(
                deployment_name = DEPLOYMENT_PREFIX + 'ui',
				provision_configuration = ui_configuration)
result = ui_deployment.poll_deployment_till_ready(retry_count=retry_count, sleep_interval=interval)
print 'UI endpoint: ' + result['componentInfo'][0]['networkEndpoints']['UserInterface']

#### 6.Turtle deployments

In [ ]:
if not wait_till_package_ready(turtle_ids['packageId'], retry_count, interval):
    exit(0)
turtle_package = client.get_package(turtle_ids['packageId'])
turtle_deployments = []
for i in range(TURTLE_NUM):
    turtle_configuration = turtle_package.get_provision_configuration(turtle_ids['planId'])
    turtle_configuration.add_routed_networks(routed_networks)
    turtle_configuration.set_component_alias('turtle', 'turtle'+str(i))
    turtle_deployment = turtle_package.provision(
                    deployment_name = DEPLOYMENT_PREFIX + 'turtle' + str(i),
                    provision_configuration = turtle_configuration)
    turtle_deployments.append(turtle_deployment)

for i in range(TURTLE_NUM):
    turtle_deployments[i].poll_deployment_till_ready(retry_count=retry_count, sleep_interval=interval)
    print('Turtle' + str(i) + ' started')

#### 7.Deprovision all

In [ ]:
retry_limit = 3
simulator_deployment.deprovision(retry_limit)
command_center_deployment.deprovision(retry_limit)
ui_deployment.deprovision(retry_limit)
for i in range(TURTLE_NUM):
    turtle_deployments[i].deprovision(retry_limit)